In [37]:
import numpy as np
import pandas as pd

import gensim
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from transformers import TFDistilBertModel, DistilBertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.metrics import Precision, Recall
from keras.utils import to_categorical
from keras.models import load_model

import warnings
warnings.filterwarnings("ignore")


In [38]:
# Загрузка данных из CSV файла
df = pd.read_csv('./combined_data.csv')
df

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [39]:
# Загрузка стоп-слов для английского языка из библиотеки NLTK
nltk.download('stopwords')

# Добавление дополнительных стоп-слов 
stop_words = stopwords.words('english')
stop_words.extend(['escapenumber'])

# Функция для предобработки текста (Очистка от малоинформативных слов, которые редко встречаются)
def preprocess(text, join_back=True):
    result = []
    # Применение simple_preprocess из библиотеки gensim для токенизации текста
    for token in gensim.utils.simple_preprocess(text):
        # Очистка от стоп-слов
        if (
            token not in gensim.parsing.preprocessing.STOPWORDS and 
            token not in stop_words
        ):
            result.append(token)
    # Объединение токенов обратно в строку
    if join_back:
        result = " ".join(result)
    return result


# Применение функции preprocess к столбцу 'text' и создание нового столбца 'clean_text'
df['clean_text']=df['text'].apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apple\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
# Разделение данных на обучающий и тестовый наборы
train, test = train_test_split(df, test_size=0.2)

# Инициализация токенизатора с ограничением числа слов до 5000
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['clean_text'])

# Преобразование текста в последовательности чисел с использованием токенизатора
X_train = tokenizer.texts_to_sequences(train['clean_text'])
X_test = tokenizer.texts_to_sequences(test['clean_text'])

# Выравнивание последовательностей до фиксированной длины (maxlen=100)
X_train = pad_sequences(X_train, maxlen=100)
X_test = pad_sequences(X_test, maxlen=100)

# Преобразование меток в формат one-hot encoding
Y_train = to_categorical(train['label'], num_classes=2)
Y_test = to_categorical(test['label'], num_classes=2)

In [43]:
epochs_ = 5
batch_size_ = 64
# loss_ = 'categorical_crossentropy'
loss_ = 'binary_crossentropy'
# loss_ = 'sparse_categorical_crossentropy'
optimizer_ = 'adam'
# optimizer_ = tf.keras.optimizers.SGD(learning_rate=0.01)
# optimizer_ = tf.keras.optimizers.Nadam(learning_rate=0.001)
# optimizer_ = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model = Sequential()
# Cлой Embedding, который используется для преобразования целых чисел (индексов слов) в векторы фиксированной размерности
# 8000 - размер словаря (количество уникальных слов), 100 - размерность векторов, input_length=100 - длина входных последовательностей
model.add(Embedding(8000, 100, input_length=100))
# Рекурентный слой
model.add(GRU(128))
# Выходной слой с 2 классами
model.add(Dense(2, activation='softmax'))

model.compile(loss=loss_, optimizer=optimizer_, metrics=[Precision(), Recall(), 'accuracy'])

model.fit(X_train, Y_train, 
          validation_data=(X_test, Y_test),
          epochs=epochs_, batch_size=batch_size_)

Epoch 1/5
1044/1044 [==============================] - 48s 45ms/step - loss: 0.0940 - precision_6: 0.9665 - recall_6: 0.9665 - accuracy: 0.9665 - val_loss: 0.0524 - val_precision_6: 0.9841 - val_recall_6: 0.9841 - val_accuracy: 0.9841
Epoch 2/5
1044/1044 [==============================] - 48s 46ms/step - loss: 0.0403 - precision_6: 0.9872 - recall_6: 0.9872 - accuracy: 0.9872 - val_loss: 0.0463 - val_precision_6: 0.9851 - val_recall_6: 0.9851 - val_accuracy: 0.9851
Epoch 3/5
1044/1044 [==============================] - 48s 46ms/step - loss: 0.0230 - precision_6: 0.9927 - recall_6: 0.9927 - accuracy: 0.9927 - val_loss: 0.0478 - val_precision_6: 0.9862 - val_recall_6: 0.9862 - val_accuracy: 0.9862
Epoch 4/5
1044/1044 [==============================] - 48s 46ms/step - loss: 0.0163 - precision_6: 0.9947 - recall_6: 0.9947 - accuracy: 0.9947 - val_loss: 0.0553 - val_precision_6: 0.9850 - val_recall_6: 0.9850 - val_accuracy: 0.9850
Epoch 5/5
1044/1044 [==============================] - 49s 4

In [ ]:
model.save('trained_model.h5')

In [ ]:
loaded_model = load_model('trained_model.h5')

# Пример сообщений для проверки обученной модели
new_data = [
    """ Subject: Get Rich Quick with Our Exclusive Investment Opportunity!

        Dear [Your Name],

        Congratulations! You've been selected for a once-in-a-lifetime investment opportunity that guarantees huge returns. Our secret system will make you rich in no time! Just click the link below to get started.

        [Spammy Link]

        Best Regards,
        Investment Master
    """,
    """ 
    Subject: Follow-Up on Our Recent Meeting

        Hi [Your Name],

        I hope this email finds you well. It was a pleasure meeting you last week. I wanted to follow up on our discussion about the upcoming project. Attached, you'll find the detailed project plan we talked about.

        Please review it at your earliest convenience, and let me know if you have any questions or concerns. I'm looking forward to our continued collaboration.

        Best Regards,
        [Sender's Full Name]

    """,
   
]

# Предобработка новых данных
preprocessed_new_data = [preprocess(text) for text in new_data]
X_new = tokenizer.texts_to_sequences(preprocessed_new_data)
X_new = pad_sequences(X_new, maxlen=100)

# Предсказание меток классов для новых данных с использованием загруженной модели
predictions = loaded_model.predict(X_new)

# Вывод результатов предсказания
for i, prediction in enumerate(predictions):
    class_label = "Спам" if prediction[1] > prediction[0] else "Не спам"
    print(f"Пример № {i+1}: Предсказанный класс - {class_label}  |  Проценты принадлежности к классам - {prediction}")


1/1 [==============================] - 0s 273ms/step
Пример № 1: Предсказанный класс - Спам  |  Проценты принадлежности к классам - [0.22153178 0.77846825]
Пример № 2: Предсказанный класс - Спам  |  Проценты принадлежности к классам - [0.45929033 0.5407096 ]
